In [ ]:
pip install contentful

In [ ]:
CTF_SPACE_ID='kg2cianb49uf'
CTF_ACCESS_TOKEN='unkkBwsQP3K7qaFKRlTqvPqspLAjPWCaUOEtiTn_Jzo'

In [ ]:
import contentful
import json
client = contentful.Client(CTF_SPACE_ID, CTF_ACCESS_TOKEN)
fields = client.entries({'content_type': 'statisticsField'})

for field in fields:
  field_id = field.fields()['field_id']
  print(field_id)

# print(client.entries({'content_type': 'statisticsField'}))

economy
administrativefinancial
population
safetyenvironment
educationsports
tourism
energy
miningindustry
agriculture
landweather
construction
international
socialsecurity
commercial
laborwage


# estatCart一覧

In [ ]:
cards = client.entries({'content_type': 'estatCardConfig'})

for card in cards:
  governmentType = card.fields()['government_type']
  params = card.fields()['estat_params']
  print(governmentType,params)

prefecture {'cdCat01': ['A1101', 'A110101', 'A110102'], 'statsDataId': '0000010101'}
prefecture {'cdCat01': ['A3200', 'A3201', 'A3202', 'A3203', 'A3204'], 'statsDataId': '0000010101'}
prefecture {'statsDataId': '0000010103', 'cdCat01': ['C121101']}
prefecture {'cdCat01': ['C6201', 'C620101', 'C620105', 'C620106', 'C620107', 'C620108', 'C620109', 'C620110', 'C620111', 'C620112', 'C620113', 'C620114', 'C620115', 'C620116', 'C620117', 'C620118', 'C620119', 'C620120', 'C620121', 'C620122'], 'statsDataId': '0000010103'}
prefecture {'cdCat01': ['C2201', 'C2206', 'C2204', 'C2205'], 'statsDataId': '0000010103'}
prefecture {'cdCat01': ['C2101', 'C2106', 'C2104', 'C2105'], 'statsDataId': '0000010103'}
prefecture {'statsDataId': '0000010104', 'cdCat01': ['D1203']}
prefecture {'statsDataId': '0000010104', 'cdCat01': ['D1204']}
prefecture {'statsDataId': '0000010104', 'cdCat01': ['D1210']}
prefecture {'statsDataId': '0000010104', 'cdCat01': ['D1205']}
city {'cdCat01': ['C120110'], 'statsDataId': '0